In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import os
import hydra
from hydra import initialize_config_module, initialize, compose
from hydra.core.global_hydra import GlobalHydra
import sys
from samgeo import SamGeo2, SamGeo
import cv2
import rasterio

# use bfloat16 for the entire notebook
torch.autocast(device_type="cuda", dtype=torch.float16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True


In [ ]:
# Load the image
#tif_path = 'Benchmarking/Kristiansund/Benchmark/20010001/Jørihaugen.tif'
#geojson_path = "Benchmarking/Kristiansund/Benchmark/20010001/20010001 Jørihaugen boligområde.geojson"

#tif_path = "Benchmarking/Kristiansund/Benchmark/19870001-02/Skjærahøgda.tif"
#geojson_path = "Benchmarking/Kristiansund/Benchmark/19870001-02/19870001-02 Skjærahøgda øst II.geojson"

tif_path = "Benchmarking/Kristiansund/Benchmark/19720001/19720001 nord.tif"
geojson_path = "Benchmarking/Kristiansund/Benchmark/19720001/19720001 Fladset A (Storbakken).geojson"

with rasterio.open(tif_path) as src:
    image = src.read()
    transform = src.transform
    crs = src.crs
    print("TIFF CRS:", crs)  # Check the CRS of the TIFF

# Convert (bands, H, W) to (H, W, bands)
image_rgb = np.dstack([image[i] for i in range(image.shape[0])])
if image_rgb.shape[2] == 1:
    image_rgb = np.repeat(image_rgb, 3, axis=2)

image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)


In [ ]:
import os
import torch
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mask_generator = SAM2AutomaticMaskGenerator.from_pretrained(
    model_id="facebook/sam2-hiera-large",
    points_per_side=32,
    points_per_batch=16,
    pred_iou_thresh=0.75,
    stability_score_thresh=0.75,
    stability_score_offset=1.0,
    mask_threshold=0.0,
    box_nms_thresh=0.5,
    crop_n_layers=1,
    crop_nms_thresh=1,
    crop_overlap_ratio=0.8,
    crop_n_points_downscale_factor=1,
    point_grids=None,
    min_mask_region_area=0,
    output_mode="binary_mask",
    use_m2m=True,
    multimask_output=False
)


print("Model and mask generator initialized successfully.")


In [ ]:
sam_result = mask_generator.generate(image_rgb)
print(len(sam_result))

In [ ]:
import numpy as np
import cv2

def is_mask_inside(outer_mask, inner_mask):
    return np.all(outer_mask[inner_mask > 0])

def is_colorful_region(image, mask, saturation_threshold=20, brightness_threshold=50):
    if mask.ndim == 2:
        mask = mask.astype(np.uint8)
    else:
        raise ValueError("The mask should be a 2D binary array.")

    if mask.shape != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    mask = mask * 255
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    masked_hsv = cv2.bitwise_and(hsv_image, hsv_image, mask=mask)

    saturation = masked_hsv[..., 1]
    brightness = masked_hsv[..., 2]

    mean_saturation = cv2.mean(saturation, mask=mask)[0]
    mean_brightness = cv2.mean(brightness, mask=mask)[0]

    return mean_saturation > saturation_threshold and mean_brightness > brightness_threshold

masks_with_areas_and_bboxes = []
for i, mask in enumerate(sam_result):
    segmentation = mask['segmentation']
    if np.any(segmentation):
        area = np.sum(segmentation)
        coords = np.argwhere(segmentation)
        y_coords, x_coords = coords[:, 0], coords[:, 1]
        min_x, max_x = x_coords.min(), x_coords.max()
        min_y, max_y = y_coords.min(), y_coords.max()
        bbox = (min_x, min_y, max_x, max_y)
        masks_with_areas_and_bboxes.append((i, segmentation, area, bbox))

masks_with_areas_and_bboxes.sort(key=lambda x: x[2], reverse=True)

contained_mask_threshold = int(0.1 * len(masks_with_areas_and_bboxes))
indices_to_remove = set()

for i, (outer_idx, outer_mask, outer_area, outer_bbox) in enumerate(masks_with_areas_and_bboxes):
    outer_min_x, outer_min_y, outer_max_x, outer_max_y = outer_bbox
    contained_count = 0
    for inner_idx, inner_mask, inner_area, inner_bbox in masks_with_areas_and_bboxes[i+1:]:
        inner_min_x, inner_min_y, inner_max_x, inner_max_y = inner_bbox
        if (inner_min_x >= outer_min_x and inner_max_x <= outer_max_x and
            inner_min_y >= outer_min_y and inner_max_y <= outer_max_y):
            if is_mask_inside(outer_mask, inner_mask):
                contained_count += 1

    if contained_count >= contained_mask_threshold:
        indices_to_remove.add(outer_idx)

filtered_masks_with_areas_and_bboxes = [
    (idx, mask, area, bbox)
    for idx, mask, area, bbox in masks_with_areas_and_bboxes
    if idx not in indices_to_remove
]

image_area = image_bgr.shape[0] * image_bgr.shape[1]
filtered_masks_with_areas_and_bboxes = [
    (idx, mask, area, bbox)
    for idx, mask, area, bbox in filtered_masks_with_areas_and_bboxes
    if area < image_area
]

filtered_masks_with_areas_and_bboxes = [
    (idx, mask, area, bbox)
    for idx, mask, area, bbox in filtered_masks_with_areas_and_bboxes
    if is_colorful_region(image_bgr, mask, saturation_threshold=20, brightness_threshold=50)
]

filtered_sam_result = [sam_result[idx] for idx, _, _, _ in filtered_masks_with_areas_and_bboxes]

print(f"Total masks after filtering: {len(filtered_sam_result)}")


In [ ]:
from shapely.geometry import MultiPolygon, Polygon

polygons_list = []
image_with_polygons = image_bgr.copy()
image_area = image_bgr.shape[0] * image_bgr.shape[1]
mask_polygons = []

def smooth_contour(contour, window_size=5):
    if window_size % 2 == 0:
        window_size += 1
    half_window = window_size // 2
    contour = np.concatenate((contour[-half_window:], contour, contour[:half_window]), axis=0)
    smoothed_contour = []
    for i in range(half_window, len(contour) - half_window):
        window_points = contour[i - half_window:i + half_window + 1]
        mean_point = np.mean(window_points, axis=0)
        smoothed_contour.append(mean_point)
    smoothed_contour = np.array(smoothed_contour, dtype=np.int32)
    return smoothed_contour

for idx, mask_dict in enumerate(filtered_sam_result):
    mask = mask_dict['segmentation'].astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        continue

    for contour in contours:
        if contour.shape[0] < 5:
            continue
        contour = contour.reshape(-1, 2)
        smoothed_contour = smooth_contour(contour, window_size=15)
        if smoothed_contour.shape[0] >= 3:
            polygon = Polygon(smoothed_contour)
            if not polygon.is_valid or polygon.area == 0:
                polygon = polygon.buffer(0)
                if not polygon.is_valid or polygon.area == 0:
                    continue
            mask_polygons.append({'area': polygon.area, 'polygon': polygon, 'index': idx})

max_area_threshold = 0.9 * image_area
mask_polygons = [mp for mp in mask_polygons if mp['area'] < max_area_threshold]

mask_polygons.sort(key=lambda x: x['area'], reverse=True)
filtered_polygons = []

def is_polygon_mostly_within(poly, existing_polys, area_overlap_threshold=0.05):
    for existing_poly in existing_polys:
        intersection_area = poly.intersection(existing_poly).area
        if poly.area == 0:
            continue
        overlap_ratio = intersection_area / poly.area
        if overlap_ratio >= area_overlap_threshold:
            return True
    return False

for idx, poly_dict in enumerate(mask_polygons):
    poly = poly_dict['polygon']
    if not is_polygon_mostly_within(poly, [d['polygon'] for d in filtered_polygons], area_overlap_threshold=0.05):
        filtered_polygons.append(poly_dict)
    else:
        print(f"Polygon {idx} is mostly within another polygon and will be removed.")

print(f"Total polygons after overlap filtering: {len(filtered_polygons)}")

for poly_dict in filtered_polygons:
    poly = poly_dict['polygon']
    if isinstance(poly, Polygon):
        coords = np.array(list(poly.exterior.coords)).astype(np.int32)
        cv2.polylines(image_with_polygons, [coords], isClosed=True, color=(0, 255, 0), thickness=10)
        polygons_list.append(poly)
    elif isinstance(poly, MultiPolygon):
        for sub_poly in poly.geoms:
            if sub_poly.is_valid and not sub_poly.is_empty:
                coords = np.array(list(sub_poly.exterior.coords)).astype(np.int32)
                cv2.polylines(image_with_polygons, [coords], isClosed=True, color=(0, 255, 0), thickness=10)
                polygons_list.append(sub_poly)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(image_with_polygons, cv2.COLOR_BGR2RGB))
plt.title("Image with Filtered Polygons (Smoothed Contours)")
plt.axis("off")
plt.show()


In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
import cv2
import matplotlib.pyplot as plt

# Assume geojson_path, transform, crs, image_bgr, image_rgb, filtered_polygons have been defined.

gdf = gpd.read_file(geojson_path)
print("Original GeoJSON CRS:", gdf.crs)
print("TIFF CRS:", crs)

if gdf.crs is None:
    gdf = gdf.set_crs("EPSG:4326", allow_override=True)



if gdf.crs != crs:
    gdf = gdf.to_crs(crs)
    print("Reprojected GeoJSON CRS:", gdf.crs)

def map_to_pixel(x_map, y_map, transform):
    col, row = ~transform * (x_map, y_map)
    return round(col), round(row)

# Convert ground-truth polygons to pixel space
gt_polygons_pixel = []
for geom in gdf.geometry:
    if geom.is_valid and geom.area > 0:
        exterior_coords = []
        for x_map, y_map in np.array(geom.exterior.coords):
            col, row = map_to_pixel(x_map, y_map, transform)
            if -50 <= col < image_rgb.shape[1] + 50 and -50 <= row < image_rgb.shape[0] + 50:
                exterior_coords.append((col, row))
        if len(exterior_coords) > 2:
            pixel_poly = Polygon(exterior_coords).buffer(0)
            if pixel_poly.is_valid and pixel_poly.area > 0:
                gt_polygons_pixel.append(pixel_poly)

# Suppress shapely RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in intersection")

def polygon_iou(poly1, poly2):
    poly1 = poly1.convex_hull.simplify(50, preserve_topology=True).buffer(100)
    poly2 = poly2.convex_hull.simplify(50, preserve_topology=True).buffer(100)
    inter_area = poly1.intersection(poly2).area
    union_area = poly1.union(poly2).area
    return inter_area / union_area if union_area > 0 else 0.0

def polygon_coverage(poly, union_poly):
    if union_poly is not None and poly.is_valid and poly.area > 0:
        intersection_area = poly.intersection(union_poly).area
        return intersection_area / poly.area
    return 0.0

predicted_polygons = [fp['polygon'] for fp in filtered_polygons]

gt_union = unary_union(gt_polygons_pixel) if gt_polygons_pixel else None
pred_union = unary_union(predicted_polygons) if predicted_polygons else None

# --------------------------
# Predicted Perspective
pred_iou_cov = []
for i, pred_poly in enumerate(predicted_polygons):
    best_iou = 0.0
    for gt_poly in gt_polygons_pixel:
        iou = polygon_iou(gt_poly, pred_poly)
        if iou > best_iou:
            best_iou = iou
    cov = polygon_coverage(pred_poly, gt_union) if best_iou > 0 else 0.0

    # Only append if not both iou and coverage are zero
    if not (best_iou == 0.0 and cov == 0.0):
        pred_iou_cov.append((i, best_iou, cov))

pred_iou_nonzero = [x[1] for x in pred_iou_cov if x[1] > 0]
pred_cov_nonzero = [x[2] for x in pred_iou_cov if x[1] > 0]

mean_iou_pred = np.mean(pred_iou_nonzero) if pred_iou_nonzero else 0.0
mean_coverage_pred = np.mean(pred_cov_nonzero) if pred_cov_nonzero else 0.0

filtered_iou_pred = [iou for (idx, iou, cov) in pred_iou_cov if iou > 0 and cov >= 0.5]
mean_iou_pred_50cov = np.mean(filtered_iou_pred) if filtered_iou_pred else 0.0

# Additional Mean IOU for IoU > 0.5
iou_above_05_pred = [iou for (idx, iou, cov) in pred_iou_cov if iou > 0.5]
mean_iou_pred_05 = np.mean(iou_above_05_pred) if iou_above_05_pred else 0.0

print("From Predicted Polygon Perspective:")
for idx, iou_val, cov_val in pred_iou_cov:
    print(f"Pred Polygon {idx}: IOU = {iou_val:.4f}, Coverage = {cov_val:.4f}")
print(f"\nMean IOU (non-zero): {mean_iou_pred:.4f}\nMean Coverage (non-zero): {mean_coverage_pred:.4f}, "
      f"\nMean IOU (Coverage≥0.5): {mean_iou_pred_50cov:.4f}\nMean IOU (IoU>0.5): {mean_iou_pred_05:.4f}")

# --------------------------
# Ground-Truth Perspective
gt_iou_cov = []
for i, gt_poly in enumerate(gt_polygons_pixel):
    best_iou = 0.0
    for pred_poly in predicted_polygons:
        iou = polygon_iou(gt_poly, pred_poly)
        if iou > best_iou:
            best_iou = iou

    cov = 0.0
    if best_iou > 0 and pred_union is not None and gt_poly.area > 0:
        intersection_area = gt_poly.intersection(pred_union).area
        cov = intersection_area / gt_poly.area

    # Append all ground-truth polygons, even if zero
    gt_iou_cov.append((i, best_iou, cov))

gt_iou_nonzero = [x[1] for x in gt_iou_cov if x[1] > 0]
gt_cov_nonzero = [x[2] for x in gt_iou_cov if x[1] > 0]

mean_iou_gt = np.mean(gt_iou_nonzero) if gt_iou_nonzero else 0.0
mean_coverage_gt = np.mean(gt_cov_nonzero) if gt_cov_nonzero else 0.0

filtered_iou_gt = [iou for (idx, iou, cov) in gt_iou_cov if iou > 0 and cov >= 0.5]
mean_iou_gt_50cov = np.mean(filtered_iou_gt) if filtered_iou_gt else 0.0

# Additional Mean IOU for IoU > 0.5 (GT perspective)
iou_above_05_gt = [x[1] for x in gt_iou_cov if x[1] > 0.5]
mean_iou_gt_05 = np.mean(iou_above_05_gt) if iou_above_05_gt else 0.0

print("\nFrom Ground-Truth Polygon Perspective:")
for idx, iou_val, cov_val in gt_iou_cov:
    print(f"GT Polygon {idx}: IOU = {iou_val:.4f}, Coverage = {cov_val:.4f}")
print(f"\nMean IOU (non-zero): {mean_iou_gt:.4f}\nMean Coverage (non-zero): {mean_coverage_gt:.4f}"
      f"\nMean IOU (Coverage≥0.5): {mean_iou_gt_50cov:.4f}\nMean IOU (IoU>0.5): {mean_iou_gt_05:.4f}")

# --------------------------
# Visualization Code
image_predicted_only = image_bgr.copy()
image_ground_truth_only = image_bgr.copy()

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 5
font_color_pred = (0, 0, 0)
font_color_gt = (0, 0, 0)
thickness = 5

# Draw predicted polygons (Green) and label them
for idx, fp in enumerate(filtered_polygons):
    polygon = fp['polygon']
    if polygon.geom_type == 'MultiPolygon':
        for poly in polygon.geoms:
            coords = np.array(list(poly.exterior.coords)).astype(np.int32)
            cv2.polylines(image_predicted_only, [coords], True, (0,255,0), thickness=10)
            centroid = poly.centroid
            cx, cy = int(centroid.x), int(centroid.y)
            cv2.putText(image_predicted_only, str(idx), (cx, cy), font, font_scale, font_color_pred, thickness, cv2.LINE_AA)
    else:
        coords = np.array(list(polygon.exterior.coords)).astype(np.int32)
        cv2.polylines(image_predicted_only, [coords], True, (0,255,0), thickness=10)
        centroid = polygon.centroid
        cx, cy = int(centroid.x), int(centroid.y)
        cv2.putText(image_predicted_only, str(idx), (cx, cy), font, font_scale, font_color_pred, thickness, cv2.LINE_AA)

# Draw ground-truth polygons (Red) and label them
for idx, gt_poly in enumerate(gt_polygons_pixel):
    if gt_poly.geom_type == 'MultiPolygon':
        for poly in gt_poly.geoms:
            coords = np.array(list(poly.exterior.coords)).astype(np.int32)
            cv2.polylines(image_ground_truth_only, [coords], True, (0,0,255), thickness=10)
            centroid = poly.centroid
            cx, cy = int(centroid.x), int(centroid.y)
            cv2.putText(image_ground_truth_only, str(idx), (cx, cy), font, font_scale, font_color_gt, thickness, cv2.LINE_AA)
    else:
        coords = np.array(list(gt_poly.exterior.coords)).astype(np.int32)
        cv2.polylines(image_ground_truth_only, [coords], True, (0,0,255), thickness=10)
        centroid = gt_poly.centroid
        cx, cy = int(centroid.x), int(centroid.y)
        cv2.putText(image_ground_truth_only, str(idx), (cx, cy), font, font_scale, font_color_gt, thickness, cv2.LINE_AA)

fig, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(cv2.cvtColor(image_predicted_only, cv2.COLOR_BGR2RGB))
axes[0].set_title("Predicted Polygons (Green) with Indices")
axes[0].axis("off")

axes[1].imshow(cv2.cvtColor(image_ground_truth_only, cv2.COLOR_BGR2RGB))
axes[1].set_title("Ground-Truth Polygons (Red) with Indices")
axes[1].axis("off")

plt.tight_layout()
plt.show()
